<a href="https://colab.research.google.com/github/Deepu-Pk/AI-Lab4/blob/main/1%3Ab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

N = 3

M = 2

class Node:
    def __init__(self, info):  # initialization node's properties
        self.info = info
        self.h = (info[0] + info[1]) / (M)

class initialization():
    # initialize the problem data
    def __init__(self):
        # Node(N, N, 0) = 3 missionaries, 3 cannibals, the boat is going to the right bank;
        # Node(0, 0, 1) = 0 missionaries, 0 cannibals, the boat is going to the left bank bank.
        self.nodes = [Node((N, N, 0)), Node((0, 0, 1))]
        self.start_node = self.nodes[0]     # Node type object
        self.goal_node = self.nodes[1].info  # --- without h


### A* classes
class CurrentNode:
    problem = None

    def __init__(self, graph_node, parent=None, g=0, f=None):
        # initialization of the current node's properties
        self.graph_node = graph_node  # Node object
        self.parent = parent		# Node object
        self.g = g					# A* function = cost of the path: root -> current node
        if f is None:
            self.f = self.g + self.graph_node.h  # FORMULA FROM A* ALGORITHM
        else:
            self.f = f

    def tree_path(self):   # find the path from current node to root
       
        current_node = self  # current node
        path = [current_node]

        while current_node.parent is not None:
            path = [current_node.parent] + path  
            current_node = current_node.parent  
        return path

    def expand(self):# find all the current node's successors
        successors = []  # LIST
        missionary_start_bank, cannibal_start_bank, boat = self.graph_node.info
        missionary_destination_bank, cannibal_destination_bank = N - missionary_start_bank, N - cannibal_start_bank

        for missionary_transport in range(M + 1):
            for cannibal_transport in range(M + 1):
                # the capacity of the boat can't be exceeded
                if missionary_transport + cannibal_transport > M:
                    continue
                # the boat must not be empty
                if missionary_transport + cannibal_transport == 0:
                    continue
                # missionaries must be >= cannibals
                if missionary_transport and cannibal_transport > missionary_transport:
                    continue

             # if the boat is leaving the left bank to go to right bank
                if boat == 0:
                    # update the number of missionaries and cannibals on the left and on the right bank
                    new_missionary_start_bank = missionary_start_bank - missionary_transport
                    new_cannibal_start_bank = cannibal_start_bank - cannibal_transport
                    new_missionary_destination_bank = missionary_destination_bank + missionary_transport
                    new_cannibal_destination_bank = cannibal_destination_bank + cannibal_transport
                else:  # if the boat is leaving the right bank to go to left bank
                    # update the number of missionaries and cannibals on the left and on the right bank
                    new_missionary_start_bank = missionary_start_bank + missionary_transport
                    new_cannibal_start_bank = cannibal_start_bank + cannibal_transport
                    new_missionary_destination_bank = missionary_destination_bank - missionary_transport
                    new_cannibal_destination_bank = cannibal_destination_bank - cannibal_transport

                if new_missionary_start_bank and new_cannibal_start_bank > new_missionary_start_bank:
                    # If there are missionaries on the bank after you have made a transport and the number of cannibals on the left bank is is higher than the number of missionaries there
                    # Then do not consider this variant
                    continue

                if new_missionary_destination_bank and new_cannibal_destination_bank > new_missionary_destination_bank:
                    # If there are missionaries on the bank after you have made a transport and the number of cannibals on the right bank is is higher than the number of missionaries there
                    # Then do not consider this variant
                    continue

                new_info = (new_missionary_start_bank,
                            new_cannibal_start_bank, 1 - boat)

                successors.append((Node(new_info), 1))

        return successors

    def check_goal_node(self):
        return self.graph_node.info == self.problem.goal_node

    def __str__(self):
        parent = self.parent if self.parent is None else self.parent.graph_node.info
        return f"\n--> : {self.graph_node.info}, parent : {parent}, h={self.graph_node.h}, g={self.g}, f={self.f}"


def DisplayInfoNodes(l):# display node's information
    s = " "
    for x in l:
        s += str(x) + "  "
    s += " "
    return s

def in_list(List, node):# check if the node is in List

    for i in range(len(List)):
        if List[i].graph_node.info == node.info:
            return List[i]
    return None


def A_star_Algorithm():
    # find the cheapest path from root (initial configuration) to goal node (final configuration).
    root = CurrentNode(CurrentNode.problem.start_node)
    open = [root]		# discovered nodes that have not been expanded yet!!
    closed = []			# discovered and expanded nodes
    # both lists contain CurrentNode type objects

    while open:  # while we have discovered nodes
        # remove the node from open
        current_node = open.pop(0)
    # add the node in closed
        closed.append(current_node)

        if current_node.check_goal_node():  # we found the final configuration
            break

        path = current_node.tree_path()

        for succesor, cost in current_node.expand():
            if in_list(path, succesor):
                continue

            # search the open_node in open list
            open_node = in_list(open, succesor)
            # search the closed_node in closed list
            closed_node = in_list(closed, succesor)

            g_new = current_node.g + cost  # = distance to successor

            if open_node:  # if the open_node is in open list
                # if we found a better distance => UPDATE
                if g_new < open_node.g:
                    open_node.g = g_new
                    open_node.f = g_new + open_node.graph_node.h
                    open_node.parent = current_node

            elif closed_node:  # if the closed_node is in closed list => calculate f
                f_new = g_new + closed_node.graph_node.h
                # if we found a better distance => UPDATE
                if f_new < closed_node.f:
                    closed_node.g = g_new
                    closed_node.f = f_new + closed_node.graph_node.h
                    closed_node.parent = current_node
                    # MOVE the node in open list to expand again when is necessary
                    open.append(closed_node)
            else:
                # the node is neither in open nor in closed => create e new node and add it to open list
                new_node = CurrentNode(
                    graph_node=succesor, parent=current_node, g=g_new)

                open.append(new_node)
        # sort by the value of function => always pop the minimum
        open.sort(key=lambda node: node.f)


    if(len(open) == 0):
        print("\nOpen list is empty, no path from start node to goal node")
    else:
        print("\nPath with minimum cost: " +
              DisplayInfoNodes(current_node.tree_path()))


if __name__ == "__main__":
    problem = initialization()
    CurrentNode.problem = problem
    A_star_Algorithm()




Path with minimum cost:  
--> : (3, 3, 0), parent : None, h=3.0, g=0, f=3.0  
--> : (3, 1, 1), parent : (3, 3, 0), h=2.0, g=1, f=3.0  
--> : (3, 2, 0), parent : (3, 1, 1), h=2.5, g=2, f=4.5  
--> : (3, 0, 1), parent : (3, 2, 0), h=1.5, g=3, f=4.5  
--> : (3, 1, 0), parent : (3, 0, 1), h=2.0, g=4, f=6.0  
--> : (1, 1, 1), parent : (3, 1, 0), h=1.0, g=5, f=6.0  
--> : (2, 2, 0), parent : (1, 1, 1), h=2.0, g=6, f=8.0  
--> : (0, 2, 1), parent : (2, 2, 0), h=1.0, g=7, f=8.0  
--> : (0, 3, 0), parent : (0, 2, 1), h=1.5, g=8, f=9.5  
--> : (0, 1, 1), parent : (0, 3, 0), h=0.5, g=9, f=9.5  
--> : (0, 2, 0), parent : (0, 1, 1), h=1.0, g=10, f=11.0  
--> : (0, 0, 1), parent : (0, 2, 0), h=0.0, g=11, f=11.0   
